I can do all this through him who gives me strength

In [1]:
import requests
from bs4 import BeautifulSoup
from scrapy.http import TextResponse

API 없어서 안됨

### BS4

In [2]:
def premier_table_bs4():
    df = pd.DataFrame(columns=["Position", "Club", "Played", "Won", "Drawn", "Lost", "GF", "GA", "GD", "Points"])
    req = requests.get("https://www.premierleague.com/tables")
    domain = BeautifulSoup(req.content, "html.parser")
    tables = domain.select("tr[data-compseason='79']")

    for info in tables:
        df.loc[len(df)] = {
            "Position" : info.select_one("#Tooltip span").text,
            "Club" : info.select_one("td.team span.long").text,
            "Played" : info.select_one("td:nth-of-type(4)").text,
            "Won" : info.select_one("td:nth-of-type(5)").text,
            "Drawn" : info.select_one("td:nth-of-type(6)").text,
            "Lost" : info.select_one("td:nth-of-type(7)").text,
            "GF" : info.select_one("td:nth-of-type(8)").text,
            "GA" :info.select_one("td:nth-of-type(9)").text,
            "GD" : info.select_one("td:nth-of-type(10)").text.replace("\n", ""),
            "Points" : info.select_one("td.points").text,
        }
    return df

In [3]:
premier_df_bs4 = premier_table_bs4()
premier_df_bs4

,Position,Club,Played,Won,Drawn,Lost,GF,GA,GD,Points
0,1,Manchester City,27,23,3,1,79,20,+59,72
1,2,Liverpool,28,16,9,3,65,32,+33,57
2,3,Manchester United,27,17,5,5,51,19,+32,56
3,4,Chelsea,27,16,5,6,49,23,+26,53
4,5,Tottenham Hotspur,27,15,7,5,52,24,+28,52
5,6,Arsenal,27,13,6,8,51,36,+15,45
6,7,Burnley,28,9,10,9,22,25,-3,37
7,8,Leicester City,28,9,9,10,40,41,-1,36
8,9,Everton,27,9,7,11,32,46,-14,34
9,10,Bournemouth,28,8,8,12,33,43,-10,32


와... BS4로 하는데 2시간 걸렸다 ㅠㅠ

구조 분석해서 
len()해서 나오는 수가 20이 되는걸 찾았는데...

처음에는 tableDark(1위), tableMid(챔스 & 강등), tablelight(챔스 예비), tr[data-compseason='79'] (나머지팀)
이렇게 4번 돌릴려고 했는데... len()을 하니 다 0이 나와서...

이번 크롤링에서는
처음부터 찾아들어가는게 어떤건지 감이 오지 않는다.

tr:nth-of-type(n) : n번째 child

### xpath

In [4]:
def premier_table_xpath():
    df = pd.DataFrame(columns=["Position", "Club", "Played", "Won", "Drawn", "Lost", "GF", "GA", "GD", "Points"])
    req = requests.get("https://www.premierleague.com/tables")
    response = TextResponse(req.url, body=req.text, encoding='utf-8')
    main = response.xpath('//*[@class="allTablesContainer"][1]/div[1]/div[1]/div[1]/table/tbody/tr[not(@class="expandable")]')[:20]

    for info in main:
        position = info.xpath('./td[2]/span[1]/text()').extract(),
        club = info.xpath('./td[3]/a/span[2]/text()').extract(),
        played = info.xpath('./td[4]/text()').extract(),
        won = info.xpath('./td[5]/text()').extract(),
        drawn = info.xpath('./td[6]/text()').extract(),
        lost = info.xpath('./td[7]/text()').extract(),
        gf = info.xpath('./td[8]/text()').extract(),
        ga = info.xpath('./td[9]/text()').extract(),
        gd = info.xpath('./td[10]/text()').extract(),
        point = info.xpath('./td[11]/text()').extract()
                            
        data = {
            "Position" : position,
            "Club" : club,
            "Played" : played,
            "Won" : won,
            "Drawn" : drawn,
            "Lost" : lost,
            "GF" : gf,
            "GA" : ga,
            "GD" : gd,
            "Points" : point
        }
    
        df.loc[len(df)] = data
    
    return df

In [5]:
premier_df_xpath = premier_table_xpath()
premier_df_xpath

,Position,Club,Played,Won,Drawn,Lost,GF,GA,GD,Points
0,"([1],)","([Manchester City],)","([27],)","([23],)","([3],)","([1],)","([79],)","([20],)","([ \n +59\n\n ],)",[72]
1,"([2],)","([Liverpool],)","([28],)","([16],)","([9],)","([3],)","([65],)","([32],)","([ \n +33\n\n ],)",[57]
2,"([3],)","([Manchester United],)","([27],)","([17],)","([5],)","([5],)","([51],)","([19],)","([ \n +32\n\n ],)",[56]
3,"([4],)","([Chelsea],)","([27],)","([16],)","([5],)","([6],)","([49],)","([23],)","([ \n +26\n\n ],)",[53]
4,"([5],)","([Tottenham Hotspur],)","([27],)","([15],)","([7],)","([5],)","([52],)","([24],)","([ \n +28\n\n ],)",[52]
5,"([6],)","([Arsenal],)","([27],)","([13],)","([6],)","([8],)","([51],)","([36],)","([ \n +15\n\n ],)",[45]
6,"([7],)","([Burnley],)","([28],)","([9],)","([10],)","([9],)","([22],)","([25],)","([ \n -3\n\n ],)",[37]
7,"([8],)","([Leicester City],)","([28],)","([9],)","([9],)","([10],)","([40],)","([41],)","([ \n -1\n\n ],)",[36]
8,"([9],)","([Everton],)","([27],)","([9],)","([7],)","([11],)","([32],)","([46],)","([ \n -14\n\n ],)",[34]
9,"([10],)","([Bournemouth],)","([28],)","([8],)","([8],)","([12],)","([33],)","([43],)","([ \n -10\n\n ],)",[32]


Text만 나오도록 못하겠어요ㅠㅠ

크롤링을 잘못한건가요??